In [13]:
import sys
import time
import math
import itertools

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from scipy.special import logit
from scipy.stats import norm

import tensorflow as tf
from keras import layers, models, datasets

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchinfo import summary
from torch.optim.lr_scheduler import StepLR, LambdaLR
import torch.autograd.profiler as profiler

from sklearn.model_selection import train_test_split, LeaveOneOut, StratifiedKFold, cross_val_predict
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, PowerTransformer
from sklearn.metrics import f1_score, log_loss, accuracy_score
from sklearn.linear_model import LogisticRegression

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [14]:
def calculate_metrics(model, data_tensor, labels_tensor, batch_size=1024, num_features=22):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for start_idx in range(0, len(data_tensor), batch_size):
            end_idx = min(start_idx + batch_size, len(data_tensor))
            inputs = data_tensor[start_idx:end_idx].view(-1, num_features)
            labels = labels_tensor[start_idx:end_idx]

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return accuracy, f1

In [15]:
class CustomDataLoader:
    def __init__(self, features, labels, validation_size=0.2, random_state=42, classification=True):        
        if validation_size > 0.0:
            stratify = labels if classification else None
            train_data, val_data, train_labels, val_labels = train_test_split(
                features, labels, test_size=validation_size, stratify=stratify, random_state=random_state
            )
            
            self.val_data_tensor = torch.tensor(val_data).float().to(device)
            
            if classification:
                self.val_labels_tensor = torch.tensor(val_labels).long().to(device)

            else:
                self.val_labels_tensor =torch.tensor(val_labels).float().to(device)
        else:
            train_data, train_labels = features, labels
            self.val_data_tensor, self.val_labels_tensor = None, None
        
        self.train_data_tensor = torch.tensor(train_data).float().to(device)

        if classification:
            self.train_labels_tensor = torch.tensor(train_labels).long().to(device)
        else:
            self.train_labels_tensor = torch.tensor(train_labels).float().to(device)

        torch.manual_seed(random_state)
        indices = torch.randperm(len(self.train_data_tensor))

        self.train_data_tensor = self.train_data_tensor[indices]
        self.train_labels_tensor = self.train_labels_tensor[indices]

In [336]:
def evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, scheduler, batch_size=1024, num_features=22, early_stopping_patience=10):
    best_val_loss = float('inf')
    best_epoch = 0
    patience_counter = 0

    last_epoch_y_pred = None
    last_epoch_y_t = None
    epoch_gamma = 0.20

    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()
        i = 0
        total_loss = 0
        num_items = 0
        epoch_y_pred=[]
        epoch_y_t=[]

        for start_idx in range(0, len(custom_train_loader.train_data_tensor), batch_size):
            end_idx = min(start_idx + batch_size, len(custom_train_loader.train_data_tensor))
            inputs = custom_train_loader.train_data_tensor[start_idx:end_idx].view(-1, num_features)
            labels = custom_train_loader.train_labels_tensor[start_idx:end_idx]

            optimizer.zero_grad()
            outputs = model(inputs)

            if epoch == 0:
                loss = criterion(outputs, labels, model)
                outputs = outputs[:, 0]
            else:
                outputs = outputs[:, 0]                
                loss = roc_star_loss(labels, outputs, epoch_gamma, last_epoch_y_t, last_epoch_y_pred)
                
            loss.backward()
            optimizer.step()
            scheduler.step()
            running_loss += loss.item() * len(labels)
            total_loss += loss.item() * len(labels)
            num_items += len(labels)

            epoch_y_pred.extend(outputs)
            epoch_y_t.extend(labels)

            i += 1

        last_epoch_y_pred = torch.tensor(epoch_y_pred).cuda()
        last_epoch_y_t = torch.tensor(epoch_y_t).cuda()
        epoch_gamma = epoch_update_gamma(last_epoch_y_t, last_epoch_y_pred, epoch)
    
        if epoch % 10 == 0:
            for param_group in optimizer.param_groups:
                print("Learning Rate:", param_group['lr'])

            # model.eval()
            # val_loss = 0.0
            # with torch.no_grad():
            #     for start_idx in range(0, len(custom_train_loader.val_data_tensor), batch_size):
            #         end_idx = min(start_idx + batch_size, len(custom_train_loader.val_data_tensor))
            #         val_inputs = custom_train_loader.val_data_tensor[start_idx:end_idx].view(-1, num_features)
            #         val_labels = custom_train_loader.val_labels_tensor[start_idx:end_idx]
    
            #         val_outputs = model(val_inputs)
            #         val_loss += roc_star_loss(val_outputs, val_labels, epoch_gamma, last_epoch_y_t, last_epoch_y_pred).item() * len(val_labels)
    
            avg_train_loss = running_loss / len(custom_train_loader.train_data_tensor)
            # avg_val_loss = val_loss / len(custom_train_loader.val_data_tensor)
    
            train_accuracy, train_f1 = calculate_metrics(model, custom_train_loader.train_data_tensor, custom_train_loader.train_labels_tensor, batch_size, num_features)
            val_accuracy, val_f1 = calculate_metrics(model, custom_train_loader.val_data_tensor, custom_train_loader.val_labels_tensor, batch_size, num_features)

            print(f'Epoch {epoch + 1}, Training Loss: {avg_train_loss}')

            # print(f'Epoch {epoch + 1}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}')
            print(f'Training Accuracy: {train_accuracy}, Training F1 Score: {train_f1}')
            print(f'Validation Accuracy: {val_accuracy}, Validation F1 Score: {val_f1}')
            print()
            
            # if avg_val_loss < best_val_loss:
            #     best_val_loss = avg_val_loss
            #     best_epoch = epoch + 1
            #     patience_counter = 0
            # else:
            #     patience_counter += 1
            #     if patience_counter >= early_stopping_patience:
            #         print(f'Early stopping triggered after {epoch + 1} epochs.')
            #         print(f'Best Validation Loss: {best_val_loss} from Epoch {best_epoch}')
            #         break

    if patience_counter < early_stopping_patience:
        print(f'Best Validation Loss after {num_epochs} epochs: {best_val_loss} from Epoch {best_epoch}')

In [337]:
data_dl = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv')
data_og = pd.read_csv('/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv')

data_dl = data_dl.drop(["id"], axis=1)

median_emp_length = data_og['person_emp_length'].median()
median_int_rate = data_og['loan_int_rate'].median()

data_dl['source'] = 0
data_og['source'] = 1

data = pd.concat([data_dl, data_og], ignore_index=True)

data['person_emp_length_missing'] = data['person_emp_length'].isna().astype(int)
data['loan_int_rate_missing'] = data['loan_int_rate'].isna().astype(int)

data['person_emp_length'] = data['person_emp_length'].fillna(median_emp_length)
data['loan_int_rate'] = data['loan_int_rate'].fillna(median_int_rate)

grade_mapping = {'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1}
data['loan_grade'] = data['loan_grade'].map(grade_mapping)

purpose_mapping = {
    'DEBTCONSOLIDATION': 1,
    'HOMEIMPROVEMENT': 2,
    'MEDICAL': 3,
    'PERSONAL': 4,
    'EDUCATION': 5,
    'VENTURE': 6
}
data['loan_intent'] = data['loan_intent'].map(purpose_mapping)

home_ownership_mapping = {
    'OWN': 1,
    'MORTGAGE': 2,
    'OTHER': 3,
    'RENT': 4
}
data['person_home_ownership'] = data['person_home_ownership'].map(home_ownership_mapping)

X = data.drop(["loan_status"], axis=1)
X = pd.get_dummies(X, drop_first=True)
y = data["loan_status"]

column_to_log = [
    'person_age',
    'person_income',
]

column_to_sqrt = [
    'person_emp_length',
    'loan_percent_income',
]

for col in column_to_log:
    if (X[col] <= 0).any():
        print(f"Column '{col}' contains non-positive values. Adding 1 to avoid log of non-positive numbers.")
        X[col] = np.log(X[col] + 1)
    else:
        X[col] = np.log(X[col])

for col in column_to_sqrt:
    if (X[col] < 0).any():
        print(f"Column '{col}' contains negative values. Setting negative values to NaN before applying sqrt.")
        X[col] = np.sqrt(X[col].clip(lower=0))
    else:
        X[col] = np.sqrt(X[col])

print(data.isnull().sum())
print(X.columns)
print(X.shape, y.shape)
print(X.columns.get_loc('source'))

person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
loan_status                   0
source                        0
person_emp_length_missing     0
loan_int_rate_missing         0
dtype: int64
Index(['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
       'source', 'person_emp_length_missing', 'loan_int_rate_missing',
       'cb_person_default_on_file_Y'],
      dtype='object')
(91226, 14) (91226,)
10


In [338]:
classes, counts = np.unique(y, return_counts=True)
for cls, count in zip(classes, counts):
    print(f"Class {cls}: {count} samples")


Class 0: 75768 samples
Class 1: 15458 samples


In [339]:
x_scaler = StandardScaler()
x_scaled = x_scaler.fit_transform(X)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(x_scaled.shape)

(91226, 14)


In [340]:
feature_means = x_scaled.mean(axis=0)
feature_variances = x_scaled.var(axis=0)
feature_mins = x_scaled.min(axis=0)
feature_maxs = x_scaled.max(axis=0)

feature_stats_scaled_full = pd.DataFrame({
    'Mean': feature_means,
    'Variance': feature_variances,
    'Min': feature_mins,
    'Max': feature_maxs
})

print("Mean, Variance, Min, and Max of Scaled Features:")
print(feature_stats_scaled_full)

Mean, Variance, Min, and Max of Scaled Features:
            Mean  Variance       Min        Max
0  -3.289997e-16       1.0 -1.552712   8.591255
1  -1.420680e-16       1.0 -5.315235   9.344027
2  -1.370832e-17       1.0 -1.810229   0.945472
3   3.987875e-17       1.0 -1.859550   8.913061
4   6.153166e-17       1.0 -1.591801   1.383254
5  -1.183900e-17       1.0 -4.464004   1.025387
6  -1.333446e-16       1.0 -1.513249   4.385625
7   9.327889e-16       1.0 -1.759487   4.065809
8  -5.358707e-16       1.0 -3.245038   4.413749
9   6.480297e-17       1.0 -0.943500   5.989958
10 -1.944089e-16       1.0 -0.745361   1.341632
11 -4.610980e-17       1.0 -0.099539  10.046317
12  6.729539e-17       1.0 -0.188056   5.317578
13  2.928596e-17       1.0 -0.433778   2.305326


In [341]:
class CustomLoss(nn.Module):
    def __init__(self, criterion, f1_lambda, f2_lambda, l1_lambda, l2_lambda):
        super(CustomLoss, self).__init__()
        self.criterion = criterion
        self.f1_lambda = f1_lambda
        self.f2_lambda = f2_lambda
        self.l1_lambda = l1_lambda
        self.l2_lambda = l2_lambda

    def forward(self, outputs, labels, model): 
        f1_loss = 0.0
        f2_loss = 0.0
        l1_loss = 0.0
        l2_loss = 0.0

        for name, module in model.named_modules():
            if isinstance(module, CustomActivation):
                f1_loss += (module.a ** 2).sum() + (module.b ** 2).sum()
                f2_loss += ((module.a - module.b) ** 2).sum()

            if isinstance(module, nn.Linear):
                l1_loss += torch.norm(module.weight, 1)
                l2_loss += torch.norm(module.weight, 2) ** 2

        total_loss = (self.criterion(outputs, labels)
                      + self.f1_lambda * f1_loss
                      + self.f2_lambda * f2_loss
                      + self.l1_lambda * l1_loss
                      + self.l2_lambda * l2_loss)

        return total_loss

    def regular_loss(self, outputs, labels):
        return self.criterion(outputs, labels)

In [342]:
class CustomActivation(nn.Module):
    def __init__(self, num_features, num_control_points, bias_tensor, init_identity=False):
        super(CustomActivation, self).__init__()
        self.a = nn.Parameter(torch.zeros(num_features, num_control_points))
        self.b = nn.Parameter(torch.zeros(num_features, num_control_points))
        
        self.local_bias = nn.Parameter(torch.zeros(num_features, num_control_points))
        self.global_bias = nn.Parameter(torch.zeros(1, num_features))
        # global_bias may not even be needed

        with torch.no_grad():
            repeated_bias = bias_tensor.repeat(num_features // bias_tensor.shape[0], 1)
            self.local_bias.copy_(repeated_bias)

            if init_identity:
                middle_index = num_control_points // 2
                self.a[:, middle_index] = 1.0
                self.b[:, middle_index] = 1.0

    def forward(self, x):
        x = x.unsqueeze(-1) + self.local_bias
        x = torch.where(x < 0, self.a * x, self.b * x)
        return x.sum(dim=-1) + self.global_bias

In [343]:
class CustomLinear(nn.Module):
    def __init__(self, num_features, num_outputs, init_identity=False):
        super(CustomLinear, self).__init__()
        
        if init_identity and num_features != num_outputs:
            raise ValueError("For identity initialization, num_features must equal num_outputs.")

        self.linear = nn.Linear(num_features, num_outputs, bias=True)
        
        with torch.no_grad():
            self.linear.bias.zero_()

            if init_identity:
                self.linear.weight.copy_(torch.eye(num_features, num_outputs))
            else:
                self.linear.weight.zero_()
                            
    def forward(self, x):
        return self.linear(x)

In [344]:
class TabularDenseNet(nn.Module):
    def __init__(self, input_size, output_size, num_control_points, num_layers, corr_comb_indices):
        super(TabularDenseNet, self).__init__()
        self.layers = nn.ModuleList()
        self.corr_comb_indices = corr_comb_indices

        quantiles = np.quantile(x_scaled, q=np.linspace(0, 1, num_control_points), axis=0).T
        bias_tensor = torch.tensor(quantiles)
        
        index_expanded = corr_comb_indices.unsqueeze(1).expand(-1, bias_tensor.size(1))
        bias_tensor_comb = torch.gather(bias_tensor, dim=0, index=index_expanded)
        
        if num_layers % 2 == 1:
            self.layers.append(CustomActivation(input_size, num_control_points, bias_tensor_comb, init_identity=True))
            num_layers -= 1
            input_size *= 2
            
        for i in range(num_layers):
            if i % 2 == 0:
                self.layers.append(CustomLinear(input_size, input_size, init_identity=True))
            else:
                self.layers.append(CustomActivation(input_size, num_control_points, bias_tensor_comb, init_identity=True))

            input_size *= 2
            
        self.final = CustomLinear(input_size, output_size, init_identity=False)
        
    def forward(self, x):
        combinations_tensor = x[:, self.corr_comb_indices]
        outputs = [combinations_tensor]

        for layer in self.layers:
            concatenated_outputs = torch.cat(outputs, dim=-1)
            outputs.append(layer(concatenated_outputs))

        concatenated_outputs = torch.cat(outputs, dim=-1)
        x = self.final(concatenated_outputs)
        return x

In [346]:
def roc_star_loss( _y_true, y_pred, gamma, _epoch_true, epoch_pred):
    """
    Nearly direct loss function for AUC.
    See article,
    C. Reiss, "Roc-star : An objective function for ROC-AUC that actually works."
    https://github.com/iridiumblue/articles/blob/master/roc_star.md
        _y_true: `Tensor`. Targets (labels).  Float either 0.0 or 1.0 .
        y_pred: `Tensor` . Predictions.
        gamma  : `Float` Gamma, as derived from last epoch.
        _epoch_true: `Tensor`.  Targets (labels) from last epoch.
        epoch_pred : `Tensor`.  Predicions from last epoch.
    """
    #convert labels to boolean
    y_true = (_y_true>=0.50)
    epoch_true = (_epoch_true>=0.50)

    # if batch is either all true or false return small random stub value.
    if torch.sum(y_true)==0 or torch.sum(y_true) == y_true.shape[0]: return torch.sum(y_pred)*1e-8

    pos = y_pred[y_true]
    neg = y_pred[~y_true]

    epoch_pos = epoch_pred[epoch_true]
    epoch_neg = epoch_pred[~epoch_true]

    max_pos = 15458 // 1000
    max_neg = 75768 // 1000
    # max_pos = 1000
    # max_neg = 1000

    cap_pos = epoch_pos.shape[0]
    cap_neg = epoch_neg.shape[0]
    epoch_pos = epoch_pos[torch.rand_like(epoch_pos) < max_pos/cap_pos]
    epoch_neg = epoch_neg[torch.rand_like(epoch_neg) < max_neg/cap_pos]

    ln_pos = pos.shape[0]
    ln_neg = neg.shape[0]

    if ln_pos>0 :
        pos_expand = pos.view(-1,1).expand(-1,epoch_neg.shape[0]).reshape(-1)
        neg_expand = epoch_neg.repeat(ln_pos)
        # print(y_pred.shape)
        # print(y_true.shape)
        # print(pos.shape)
        # print(ln_pos)
        # print(ln_neg)
        # print(pos_expand.shape)
        # print(epoch_neg.shape)
        # print(neg_expand.shape)

        diff2 = neg_expand - pos_expand + gamma
        l2 = diff2[diff2>0]
        m2 = l2 * l2
        len2 = l2.shape[0]
    else:
        m2 = torch.tensor([0], dtype=torch.float).cuda()
        len2 = 0

    if ln_neg>0 :
        pos_expand = epoch_pos.view(-1,1).expand(-1, ln_neg).reshape(-1)
        neg_expand = neg.repeat(epoch_pos.shape[0])

        diff3 = neg_expand - pos_expand + gamma
        l3 = diff3[diff3>0]
        m3 = l3*l3
        len3 = l3.shape[0]
    else:
        m3 = torch.tensor([0], dtype=torch.float).cuda()
        len3=0

    if (torch.sum(m2)+torch.sum(m3))!=0 :
       res2 = torch.sum(m2)/max_pos+torch.sum(m3)/max_neg
    else:
       res2 = torch.sum(m2)+torch.sum(m3)

    res2 = torch.where(torch.isnan(res2), torch.zeros_like(res2), res2)

    return res2

In [347]:
def epoch_update_gamma(y_true,y_pred, epoch=-1,delta=2):
    """
    Calculate gamma from last epoch's targets and predictions.
    Gamma is updated at the end of each epoch.
    y_true: `Tensor`. Targets (labels).  Float either 0.0 or 1.0 .
    y_pred: `Tensor` . Predictions.
    """
    DELTA = delta
    SUB_SAMPLE_SIZE = 2000.0
    pos = y_pred[y_true==1]
    neg = y_pred[y_true==0] # yo pytorch, no boolean tensors or operators?  Wassap?
    # subsample the training set for performance
    cap_pos = pos.shape[0]
    cap_neg = neg.shape[0]
    pos = pos[torch.rand_like(pos) < SUB_SAMPLE_SIZE/cap_pos]
    neg = neg[torch.rand_like(neg) < SUB_SAMPLE_SIZE/cap_neg]
    ln_pos = pos.shape[0]
    ln_neg = neg.shape[0]
    pos_expand = pos.view(-1,1).expand(-1,ln_neg).reshape(-1)
    neg_expand = neg.repeat(ln_pos)
    diff = neg_expand - pos_expand
    ln_All = diff.shape[0]
    Lp = diff[diff>0] # because we're taking positive diffs, we got pos and neg flipped.
    ln_Lp = Lp.shape[0]-1
    diff_neg = -1.0 * diff[diff<0]
    diff_neg = diff_neg.sort()[0]
    ln_neg = diff_neg.shape[0]-1
    ln_neg = max([ln_neg, 0])
    left_wing = int(ln_Lp*DELTA)
    left_wing = max([0,left_wing])
    left_wing = min([ln_neg,left_wing])
    default_gamma=torch.tensor(0.2, dtype=torch.float).cuda()
    if diff_neg.shape[0] > 0 :
       gamma = diff_neg[left_wing]
    else:
       gamma = default_gamma # default=torch.tensor(0.2, dtype=torch.float).cuda() #zoink
    L1 = diff[diff>-1.0*gamma]
    ln_L1 = L1.shape[0]
    if epoch > -1 :
        return gamma
    else :
        return default_gamma

In [348]:
custom_train_loader = CustomDataLoader(x_scaled, y_encoded, validation_size=0.2, random_state=0, classification=True)
print(custom_train_loader.train_data_tensor.shape)

torch.Size([72980, 14])


In [349]:
interaction_order = 14
x = custom_train_loader.train_data_tensor
batch_size = x.size(0)
num_features = x.size(1)

comb_indices = list(itertools.combinations(range(num_features), interaction_order))
comb_indices = torch.tensor(comb_indices, dtype=torch.long)
corr_comb_indices = comb_indices[0]

print(corr_comb_indices.shape)
print(corr_comb_indices)

torch.Size([14])
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])


In [350]:
models = []

In [351]:
feature_tensor = torch.empty((custom_train_loader.train_data_tensor.size(0), 0))
print(feature_tensor.shape)

torch.Size([72980, 0])


In [352]:
num_og_features = 14
num_features = interaction_order
num_classes = 2
num_control_points = 15
num_epochs = 1000
batch_size = 7298

In [353]:
torch.cuda.empty_cache()

In [ ]:
def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.9999 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, num_control_points, 4, corr_comb_indices).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001 * 5 * 0.00001 * 0.01)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0001 * 0.0, 0.0001 * 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, scheduler, batch_size, num_og_features, early_stopping_patience=100)

models.append(model)
print(f"Execution time: {(time.time() - start_time):.6f} seconds")

Learning Rate: 5.000000000000001e-11
Epoch 1, Training Loss: 0.6931459903717041
Training Accuracy: 0.8064949301178405, Training F1 Score: 0.8122415151345942
Validation Accuracy: 0.8089992327085389, Validation F1 Score: 0.8140481026508355

here
Learning Rate: 4.995002249400106e-10
Epoch 11, Training Loss: 4.507810402731849e-10
Training Accuracy: 0.8150178130994793, Training F1 Score: 0.7471134511891488
Validation Accuracy: 0.8133837553436369, Validation F1 Score: 0.7457248172508865

Learning Rate: 4.94529867378049e-10
Epoch 21, Training Loss: 7.298563142743575e-10
Training Accuracy: 0.8188407782954235, Training F1 Score: 0.7482208659955182
Validation Accuracy: 0.8182067302422449, Validation F1 Score: 0.7475940072891669

Learning Rate: 4.896089681607693e-10
Epoch 31, Training Loss: 1.5208984072145881e-09
Training Accuracy: 0.8176760756371608, Training F1 Score: 0.7474882286025458
Validation Accuracy: 0.8176586649128577, Validation F1 Score: 0.7472214327206057



In [ ]:
for name, param in model.named_parameters():
    print(name)
    print(param)

In [ ]:
num_og_features = 14
num_features = interaction_order
num_classes = 2
num_control_points = 15
num_epochs = 1000
batch_size = 7298

def custom_lr_lambda(step):
    num_step_threshold = 100

    if step < num_step_threshold:
        return step / num_step_threshold
    if step == num_step_threshold:
        print("here")
    return 0.9999 ** (step - num_step_threshold)

start_time = time.time()
model = TabularDenseNet(num_features, num_classes, num_control_points, 4, corr_comb_indices).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.01 * 0.02 * 10)
scheduler = LambdaLR(optimizer, lr_lambda=custom_lr_lambda)

criterion = CustomLoss(nn.CrossEntropyLoss(), 0.0, 0.0, 0.0001 * 0.0, 0.0001 * 0.0)
evaluate_model(model, custom_train_loader, criterion, optimizer, num_epochs, scheduler, batch_size, num_og_features, early_stopping_patience=100)

models.append(model)
print(f"Execution time: {(time.time() - start_time):.6f} seconds")

In [ ]:
data = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')

data = data.drop(["id"], axis=1)
data['source'] = 0

grade_mapping = {'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1}
data['loan_grade'] = data['loan_grade'].map(grade_mapping)

purpose_mapping = {
    'DEBTCONSOLIDATION': 1,
    'HOMEIMPROVEMENT': 2,
    'MEDICAL': 3,
    'PERSONAL': 4,
    'EDUCATION': 5,
    'VENTURE': 6
}
data['loan_intent'] = data['loan_intent'].map(purpose_mapping)

home_ownership_mapping = {
    'OWN': 1,
    'MORTGAGE': 2,
    'OTHER': 3,
    'RENT': 4
}
data['person_home_ownership'] = data['person_home_ownership'].map(home_ownership_mapping)

print(data.columns)
print(data.isnull().sum())

X = data.drop([], axis=1)

X = pd.get_dummies(X, drop_first=True)

column_to_log = [
    'person_age',
    'person_income',
]

column_to_sqrt = [
    'person_emp_length',
    'loan_percent_income',
]

for col in column_to_log:
    if (X[col] <= 0).any():
        print(f"Column '{col}' contains non-positive values. Adding 1 to avoid log of non-positive numbers.")
        X[col] = np.log(X[col] + 1)
    else:
        X[col] = np.log(X[col])

for col in column_to_sqrt:
    if (X[col] < 0).any():
        print(f"Column '{col}' contains negative values. Setting negative values to NaN before applying sqrt.")
        X[col] = np.sqrt(X[col].clip(lower=0))
    else:
        X[col] = np.sqrt(X[col])

print(X.shape)
print(X.columns)

In [ ]:
print(x_scaled)

In [ ]:
print(X.shape)
X_scaled_test = x_scaler.transform(X)
print(X_scaled_test.shape)
print(X_scaled_test)

In [ ]:
X_scaled_test_tensor = torch.tensor(X_scaled_test).float().to(device)
outputs = models[-1](X_scaled_test_tensor)
print(outputs)

In [ ]:
probabilities = F.softmax(outputs, dim=1)
print(probabilities)

In [ ]:
positive_class_probs = probabilities[:, 1]
print(positive_class_probs)

In [ ]:
import pandas as pd

test_df = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
ids = test_df['id']

positive_class_probs = positive_class_probs.cpu().detach().numpy()

submission_df = pd.DataFrame({
    'id': ids,
    'loan_status': positive_class_probs
})

submission_df.to_csv('submission.csv', index=False)
print("Submission file created successfully.")